# A script for assigning boolean value to 'it is night time'

## Jason Kniss
### Feb 22 2024

Import libraries

In [1]:
# Import libraries here
import pandas as pd
import numpy as np
import netCDF4 as nc

Import data

In [ ]:
# Optionally declare Lat and Long for stationary sites
lat = 80.05 # North(+)
long = 86.42 # West(+)

In [2]:
# Optionally declare Lat and Long for stationary sites
lat = 80.05 # North(+)
long = 86.42 # West(+)

# Using a sample Eureka .txt file with a time stamp
tower_hrly = 'data/eureka-data/tower-met/eurmeteorologicaltwr.b1.20110701.000000.txt' # Tower meteorological data for July 01 2011

with open(tower_hrly, "r") as f:
    lines = f.readlines()

header = lines[0].strip().split()
data_rows = [line.strip().split("\t") for line in lines[1:]]
df = pd.DataFrame(data_rows, columns=header)
        # Rename the dataframe when more data is imported

df = df.astype('float')
df = df[df['HourMin'] != 2400] # Filter out invalid HourMin values 

# Note, I'd prefer to have this change the 2400 HourMin to 0000 the following day, but I wasn't able to figure that out.

In [3]:
df.head()

,DayFrac,Year,JulianDay,HourMin,Pressure[mbar],10MVTair[degC],10MRH[%],6MVTair[degC],6MRH[%],2MVTair[degC],...,2MTair[degC],10MTCETair[degC],6MTCETair[degC],6MTCWTair[degC],2MTCWTair[degC],Wdir[deg],Wspd[m/s],HumidityQC,AirTempQC,TCAirTempQC
0,182.000,2011.0,182.0,0.0,1006.3,18.695,22.882,18.781,23.215,18.933,...,18.346,19.155,19.320,19.481,-9999.0,176.99,2.6605,0.0,0.0,1.0
1,182.001,2011.0,182.0,1.0,1006.4,18.719,23.086,18.856,23.790,18.990,...,18.392,19.351,19.578,19.562,-9999.0,195.14,1.9895,0.0,0.0,1.0
2,182.001,2011.0,182.0,2.0,1006.4,18.802,23.171,18.892,23.560,19.018,...,18.406,19.430,19.517,19.658,-9999.0,180.96,2.0947,0.0,0.0,1.0
3,182.002,2011.0,182.0,3.0,1006.4,18.793,22.865,18.861,23.263,18.985,...,18.389,19.124,19.244,19.377,-9999.0,165.34,2.3053,0.0,0.0,1.0
4,182.003,2011.0,182.0,4.0,1006.4,18.691,22.890,18.768,23.457,18.933,...,18.341,19.227,19.421,19.453,-9999.0,160.13,2.1737,0.0,0.0,1.0


### 1) Determine declination

define sin/cos functions that expect degrees as the input

In [4]:
def cosd(degrees):
     return np.cos(np.deg2rad(degrees))
def sind(degrees):
    return np.sin(np.deg2rad(degrees))

In [5]:
df['B'] = (df['JulianDay'] - 1)*(360/365)
df['decl_1'] = (180/np.pi)*(0.006918 - 0.399912*cosd(df['B']) + 0.070257*sind(df['B']) - 0.006785*cosd(2*df['B']) + 0.000907*sind(2*df['B']) - 0.002697*cosd(3*df['B']) + 0.00148*sind(3*df['B']))
df['decl_2'] = 23.45*sind(360*(284 + df['JulianDay'])/365) # Less accurate but typically adequate for most engineering purposes

### 2) Determine hour angle

2a) determine local time in terms of minutes from *solar* noon (+ afternoon, - morning)

**Eureka data is logged as GMT/UTC time**

In [6]:
df['E'] = 229.2*(0.000075 + 0.001868*cosd(df['B']) - 0.032077*sind(df['B']) - 0.014615*cosd(2*df['B']) - 0.04089*sind(2*df['B']))

df['t_corr'] = 4*(0-long) + df['E']

In [10]:
def time_to_minutes(time_float):
    # Convert float to string and then to HHMM format
    time_str = str(int(time_float)).zfill(4)
    # Convert HHMM string to minutes
    hours = int(time_str[:2])
    minutes = int(time_str[2:])
    return hours * 60 + minutes

def minutes_from_noon(time_float):
    # Calculate the difference between the time and noon (720 minutes)
    time_minutes = time_to_minutes(time_float)
    minutes_difference = time_minutes - 720
    return minutes_difference

# Apply the minutes_from_noon function to the 'HourMin' column
df['MinutesFromNoon'] = df['HourMin'].apply(minutes_from_noon)

In [12]:
df['t_sol'] = df['MinutesFromNoon'] + df['t_corr']

In [15]:
df['w'] = df['t_sol'] * (360/1440)

In [19]:
df['theta_z'] =  np.degrees(np.arccos(cosd(lat) * cosd(df['decl_1']) * cosd(df['w']) + sind(lat) * sind(df['decl_1'])))

In [23]:
print(min(df['theta_z']))
print(max(df['theta_z']))

56.87435857252933
76.77435468412504


In [24]:
df['isnight'] = (df['theta_z'] > 105) | (df['theta_z'] < -105)

In [27]:
df.head()

,DayFrac,Year,JulianDay,HourMin,Pressure[mbar],10MVTair[degC],10MRH[%],6MVTair[degC],6MRH[%],2MVTair[degC],...,B,decl_1,decl_2,E,t_corr,MinutesFromNoon,t_sol,w,theta_z,isnight
0,182.000,2011.0,182.0,0.0,1006.3,18.695,22.882,18.781,23.215,18.933,...,178.520548,23.175644,23.120484,-3.462145,-349.142145,-720,-1069.142145,-267.285536,67.659564,False
1,182.001,2011.0,182.0,1.0,1006.4,18.719,23.086,18.856,23.790,18.990,...,178.520548,23.175644,23.120484,-3.462145,-349.142145,-719,-1068.142145,-267.035536,67.702438,False
2,182.001,2011.0,182.0,2.0,1006.4,18.802,23.171,18.892,23.560,19.018,...,178.520548,23.175644,23.120484,-3.462145,-349.142145,-718,-1067.142145,-266.785536,67.745290,False
3,182.002,2011.0,182.0,3.0,1006.4,18.793,22.865,18.861,23.263,18.985,...,178.520548,23.175644,23.120484,-3.462145,-349.142145,-717,-1066.142145,-266.535536,67.788118,False
4,182.003,2011.0,182.0,4.0,1006.4,18.691,22.890,18.768,23.457,18.933,...,178.520548,23.175644,23.120484,-3.462145,-349.142145,-716,-1065.142145,-266.285536,67.830922,False


## To Do

1) Early in the code - after import - start by converting all times from GMT to solar ti